<a href="https://colab.research.google.com/github/jumbokh/csu1112-class/blob/main/notebooks/flags/F2379_CH13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#第13章：實務上的最佳實踐

#讓模型發揮最大效用

###13-1-1 超參數優化

In [ ]:
!pip install keras-tuner -q

     |████████████████████████████████| 133 kB 5.4 MB/s 


####程式 13.1 KerasTuner模型建構函式

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

def build_model(hp):
    units = hp.Int(name="units", min_value=16, max_value=64, step=16)
    model = keras.Sequential([
        layers.Dense(units, activation="relu"),
        layers.Dense(10, activation="softmax")
    ])
    optimizer = hp.Choice(name="optimizer", values=["rmsprop", "adam"])
    model.compile(
        optimizer=optimizer,
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"])
    return model

####程式 13.2 繼承HyperModel的子類別

In [ ]:
import kerastuner as kt

class SimpleMLP(kt.HyperModel):
    def __init__(self, num_classes):
        self.num_classes = num_classes

    def build(self, hp):
        units = hp.Int(name="units", min_value=16, max_value=64, step=16)
        model = keras.Sequential([
            layers.Dense(units, activation="relu"),
            layers.Dense(self.num_classes, activation="softmax")
        ])
        optimizer = hp.Choice(name="optimizer", values=["rmsprop", "adam"])
        model.compile(
            optimizer=optimizer,
            loss="sparse_categorical_crossentropy",
            metrics=["accuracy"])
        return model

hypermodel = SimpleMLP(num_classes=10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  """Entry point for launching an IPython kernel.


In [ ]:
tuner = kt.BayesianOptimization(
    build_model,
    objective="val_accuracy",
    max_trials=100,
    executions_per_trial=2,
    directory="mnist_kt_test",
    overwrite=True,
)

In [ ]:
tuner.search_space_summary()

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 64, 'step': 16, 'sampling': None}
optimizer (Choice)
{'default': 'rmsprop', 'conditions': [], 'values': ['rmsprop', 'adam'], 'ordered': False}


In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape((-1, 28 * 28)).astype("float32") / 255
x_test = x_test.reshape((-1, 28 * 28)).astype("float32") / 255
x_train_full = x_train[:]
y_train_full = y_train[:]
num_val_samples = 10000
x_train, x_val = x_train[:-num_val_samples], x_train[-num_val_samples:]
y_train, y_val = y_train[:-num_val_samples], y_train[-num_val_samples:]
callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=5),
]
tuner.search(
    x_train, y_train,
    batch_size=128,
    epochs=100,
    validation_data=(x_val, y_val),
    callbacks=callbacks,
    verbose=2,
)

Trial 8 Complete [00h 01m 12s]
val_accuracy: 0.974700003862381

Best val_accuracy So Far: 0.9760499894618988
Total elapsed time: 00h 09m 39s

Search: Running Trial #9

Value             |Best Value So Far |Hyperparameter
64                |64                |units
rmsprop           |rmsprop           |optimizer

Epoch 1/100
391/391 - 2s - loss: 0.4324 - accuracy: 0.8828 - val_loss: 0.2422 - val_accuracy: 0.9335 - 2s/epoch - 5ms/step
Epoch 2/100
391/391 - 1s - loss: 0.2302 - accuracy: 0.9341 - val_loss: 0.1820 - val_accuracy: 0.9505 - 1s/epoch - 4ms/step
Epoch 3/100
391/391 - 2s - loss: 0.1776 - accuracy: 0.9495 - val_loss: 0.1621 - val_accuracy: 0.9552 - 2s/epoch - 4ms/step
Epoch 4/100
391/391 - 1s - loss: 0.1463 - accuracy: 0.9579 - val_loss: 0.1425 - val_accuracy: 0.9585 - 1s/epoch - 3ms/step
Epoch 5/100
391/391 - 1s - loss: 0.1247 - accuracy: 0.9640 - val_loss: 0.1231 - val_accuracy: 0.9651 - 1s/epoch - 3ms/step
Epoch 6/100
391/391 - 1s - loss: 0.1089 - accuracy: 0.9684 - val_loss: 

KeyboardInterrupt: ignored

####程式13.3 查詢最佳的超參數配置

In [ ]:
top_n = 4
best_hps = tuner.get_best_hyperparameters(top_n)

In [ ]:
def get_best_epoch(hp):
    model = build_model(hp)
    callbacks=[
        keras.callbacks.EarlyStopping(
            monitor="val_loss", mode="min", patience=10)
    ]
    history = model.fit(
        x_train, y_train,
        validation_data=(x_val, y_val),
        epochs=100,
        batch_size=128,
        callbacks=callbacks)
    val_loss_per_epoch = history.history["val_loss"]
    best_epoch = val_loss_per_epoch.index(min(val_loss_per_epoch)) + 1
    print(f"Best epoch: {best_epoch}")
    return best_epoch

In [ ]:
def get_best_trained_model(hp):
    best_epoch = get_best_epoch(hp)
    model = build_model(hp)
    model.fit(
        x_train_full, y_train_full,
        batch_size=128, epochs=int(best_epoch * 1.2))
    return model

best_models = []
for hp in best_hps:
    model = get_best_trained_model(hp)
    model.evaluate(x_test, y_test)
    best_models.append(model)

Epoch 1/100
391/391 [==============================] - 6s 9ms/step - loss: 0.4174 - accuracy: 0.8891 - val_loss: 0.2303 - val_accuracy: 0.9374
Epoch 2/100
391/391 [==============================] - 3s 7ms/step - loss: 0.2109 - accuracy: 0.9400 - val_loss: 0.1713 - val_accuracy: 0.9540
Epoch 3/100
391/391 [==============================] - 2s 6ms/step - loss: 0.1598 - accuracy: 0.9539 - val_loss: 0.1477 - val_accuracy: 0.9593
Epoch 4/100
391/391 [==============================] - 2s 4ms/step - loss: 0.1311 - accuracy: 0.9620 - val_loss: 0.1370 - val_accuracy: 0.9596
Epoch 5/100
391/391 [==============================] - 2s 4ms/step - loss: 0.1106 - accuracy: 0.9679 - val_loss: 0.1231 - val_accuracy: 0.9655
Epoch 6/100
391/391 [==============================] - 2s 4ms/step - loss: 0.0961 - accuracy: 0.9721 - val_loss: 0.1145 - val_accuracy: 0.9689
Epoch 7/100
391/391 [==============================] - 2s 4ms/step - loss: 0.0843 - accuracy: 0.9754 - val_loss: 0.1102 - val_accuracy: 0.9681

In [ ]:
best_models = tuner.get_best_models(top_n)

##13-2 擴大模型的訓練規模

###13-2-1 使用混合精度加速GPU上的訓練

In [ ]:
import tensorflow as tf
import numpy as np
np_array = np.zeros((2, 2))
tf_tensor = tf.convert_to_tensor(np_array)
tf_tensor.dtype

In [ ]:
np_array = np.zeros((2, 2))
tf_tensor = tf.convert_to_tensor(np_array, dtype="float32")
tf_tensor.dtype

tf.float32

In [ ]:
from tensorflow import keras
keras.mixed_precision.set_global_policy("mixed_float16")

The dtype policy mixed_float16 may run slowly because this machine does not have a GPU. Only Nvidia GPUs with compute capability of at least 7.0 run quickly with mixed_float16.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once


###13-2-3 在TPU上進行訓練

In [ ]:
import tensorflow as tf 
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
print("Device:", tpu.master())

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.3.51.154:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.3.51.154:8470


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


Device: grpc://10.3.51.154:8470


####程式 13.4 在TPUStrategy區塊中建立模型

In [ ]:
keras.mixed_precision.set_global_policy("float32")

from tensorflow import keras
from tensorflow.keras import layers

strategy = tf.distribute.TPUStrategy(tpu)
print(f"Number of replicas:{strategy.num_replicas_in_sync}")

def build_model(input_size):
    inputs = keras.Input((input_size, input_size, 3))
    x = keras.applications.resnet.preprocess_input(inputs)
    x = keras.applications.resnet.ResNet50(
        weights=None, include_top=False, pooling="max")(x)
    outputs = layers.Dense(10, activation="softmax")(x)
    model = keras.Model(inputs, outputs)
    model.compile(optimizer='rmsprop',
           loss="sparse_categorical_crossentropy",
           metrics=["accuracy"])
    return model

with strategy.scope():
    model = build_model(input_size=32)

In [ ]:
(x_train,y_train), (x_test,y_test) = keras.datasets.cifar10.load_data()
model.fit(x_train, y_train, batch_size=1024)

49/49 [==============================] - 75s 573ms/step - loss: 2.5839 - accuracy: 0.2814


## Summary